# Removing records on land

When you download data from OBIS, one of the first steps in the data cleaning is usually to remove records on land. A species might be recorded on land by many reasons: positional uncertainty, misidentification, records on museums or aquariums, etc.

In this short notebook we will explore how to remove records on land using two different tools:

1. The `obistools` package
2. A raster file containing environmental information

We will also explore how we can approximate records on land (what might be useful in situations like intertidal species).

## The `obistools` package

The `obistools` R package provide tools for data enhancement and quality control. You can find it [here]() and install using:

`devtools::install_github("iobis/obistools")`

Many functions are intended for improving data **before** submitting it to OBIS, but others can be used after data download. We will specifically use the function `obistools::check_onland()`

You need to supply only one argument (`data`), a `data.frame`  with columns `decimalLongitude` and `decimalLatitude`. Additionally you can supply your own polygon for the land (in `sf` format).

We will first download some data to play with. We will use the fiddler crab _Minuca rapax_ as an example.

In [ ]:
library(obistools)
library(patchwork)
library(ggplot2)

crab <- robis::occurrence("Minuca rapax")

head(crab)

plot_map(crab)

Now we can check which records are on land.

In [ ]:
crab_on_land <- check_onland(crab, report = TRUE)
print(crab_on_land)
nrow(crab_on_land)
nrow(crab)


Almost all records, but that is expected for an intertidal species. That's why we can also consider a buffer. But first, let's see how we can remove those records on land:

In [ ]:
crab_on_sea <- crab[-1 * crab_on_land$row,]
nrow(crab_on_sea)

The function also has a `buffer` argument, expressed in meters from the shoreline. Let's try with a buffer of 200m

In [ ]:
crab_on_land_buff <- check_onland(crab, report = TRUE, buffer = 200)
nrow(crab_on_land_buff)
nrow(crab)

A big difference. Let's also produce a version with those records removed.

In [ ]:
crab_on_sea_buff <- crab[-1 * crab_on_land_buff$row,]
nrow(crab_on_sea_buff)

In [ ]:
p1 <- plot_map(crab) + ggtitle("Original") + coord_sf(xlim = c(-100, -20))
p2 <- plot_map(crab_on_sea) + ggtitle("No buffer") + coord_sf(xlim = c(-100, -20))
p3 <- plot_map(crab_on_sea_buff) + ggtitle("Buffer") + coord_sf(xlim = c(-100, -20))

p1+p2+p3+plot_layout(nrow = 1)

## Using a raster layer

We can also use a raster layer to remove records on land. This is actually a very easy way to clean up the data and ensure you have only points for which you have coverage in your environmental dataset.

We start by downloading a raster layer. We will download an SST layer from the NOAA CoralTemp product available [here](https://coastwatch.pfeg.noaa.gov/erddap/files/NOAA_DHW_monthly/).

All processing will be done using the package `terra`, but you can achieve the same results with package `stars`.

In [ ]:
library(terra)

# Download file
download.file("https://coastwatch.pfeg.noaa.gov/erddap/files/NOAA_DHW_monthly/ct5km_sst_ssta_monthly_v31_202401.nc",
"sst_202401.nc", method = "wget")

In [ ]:
sst <- rast("sst_202401.nc")

sst

sst <- sst$sea_surface_temperature

plot(sst)

To remove records with no valid data, we just need to extract the values according to the longitude and latitude.

In [ ]:
# It is always interesting to add terra:: before the extract function
# because extract is a common name for function on the tidyverse and some other packages.
crab_extract <- terra::extract(sst, crab[,c("decimalLongitude", "decimalLatitude")])

# If you encounter an error, convert crab[,c("decimalLongitude", "decimalLatitude")] into
# a data.frame using data.frame(crab[,c("decimalLongitude", "decimalLatitude")])

head(crab_extract)

We see that we have many NAs, that is points for which there is no environmental information. We remove those.

In [ ]:
crab_on_sea_sst <- crab[!is.na(crab_extract$sea_surface_temperature),]

nrow(crab_on_sea_sst)

In [ ]:
# Plot
crab_plot <- crab
crab_plot$status <- ifelse(is.na(crab_extract$sea_surface_temperature), "On land", "On sea")

sst_crop <- crop(sst, ext(vect(crab_plot, geom = c("decimalLongitude", "decimalLatitude"))))
sst_crop <- as.data.frame(sst_crop, xy = T)

ggplot() +
    geom_raster(data = sst_crop, aes(x = x, y = y, fill = sea_surface_temperature)) +
    geom_point(data = crab_plot,
     aes(x = decimalLongitude, y = decimalLatitude, 
     color = status), size = 2) +
    scale_fill_viridis_c() +
    scale_color_manual(values = c("#120101", "#b100cc")) +
    theme_light() +
    coord_equal()


## Approximating points to land

We can use that same raster layer to approximate points to land, that is, bring an invalid point to the closest valid cell.

There are many ways of doing that, and in fact you can also approximate points to the nearest point on a vector (shoreline). Here we will use a very simple approach based on nearby cells.

We start by getting the cell indices of the NA points. Then we see if any of the adjacent cells is valid (using the queen movement, which looks into the 8 adjacent cells). If so, we chose the one that is geographically close (considering distance).

In [ ]:
crab$ID <- 1:nrow(crab)
coordnames <- c("decimalLongitude", "decimalLatitude")

non_valid_coord <- crab[is.na(crab_extract$sea_surface_temperature), c(coordnames, "ID")]

for (i in seq_len(nrow(non_valid_coord))) {
    tcell <- cellFromXY(sst, data.frame(non_valid_coord[i,1:2]))
    adj_cells <- adjacent(sst, tcell, "queen")
    adj_values <- terra::extract(sst, as.vector(adj_cells))
    valid_cells <- adj_cells[!is.na(as.vector(adj_values[,1]))]
    if (length(valid_cells) > 0) {
        if (length(valid_cells) == 1) {
            non_valid_coord[i, coordnames] <- xyFromCell(sst, valid_cells)
        } else {
            valid_xy <- xyFromCell(sst, valid_cells)
            valid_dists <- distance(
                vect(data.frame(non_valid_coord[i,1:2]), geom = coordnames, crs = "EPSG:4326"),
                vect(data.frame(valid_xy), geom = c("x", "y"), crs = "EPSG:4326")
            )
            nearest_pt <- valid_xy[order(valid_dists),][1,]
            non_valid_coord[i, coordnames] <- data.frame(x = nearest_pt[1], y = nearest_pt[2])
        }
    } 
}

In [ ]:
crab_merged <- merge(crab, non_valid_coord, by = "ID", all.x = TRUE, suffixes = c("", "_updated"))

crab_merged$decimalLongitude <- ifelse(is.na(crab_merged$decimalLongitude_updated),
    crab_merged$decimalLongitude,
    crab_merged$decimalLongitude_updated)
crab_merged$decimalLatitude <- ifelse(is.na(crab_merged$decimalLatitude_updated),
    crab_merged$decimalLatitude,
    crab_merged$decimalLatitude_updated)

crab_merged <- crab_merged[, !grepl("_updated", colnames(crab_merged))]

head(crab_merged)

In [ ]:
# Extract the data again
crab_extract_prox <- terra::extract(sst, crab_merged[,c("decimalLongitude", "decimalLatitude")])

crab_on_sea_sst_prox <- crab[!is.na(crab_extract_prox$sea_surface_temperature),]

nrow(crab_on_sea_sst_prox)